In [3]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

In [4]:
from braket.aws import AwsDevice
from braket.circuits import Circuit, gates, noises, observables
from braket.devices import LocalSimulator
from braket.parametric import FreeParameter
import numpy as np
from scipy.stats import unitary_group

Make Circuit

In [45]:
# build a simple circuit
circ = Circuit().h(0).cnot(0,1)

Noise Testing Suite

In [46]:
# define a noise channel
noise = noises.BitFlip(probability=0.1)

# add noise to every gate in the circuit
circ.apply_gate_noise(noise)

Circuit('instructions': [Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': BitFlip(0.1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': CNot('qubit_count': 2), 'target': QubitSet([Qubit(0), Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': BitFlip(0.1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': BitFlip(0.1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)])

Run noise test

In [47]:
#noise add

# select the local noise simulator
device = LocalSimulator('braket_dm')

# run the circuit on the local simulator
task = device.run(circ, shots = 1000)

# visualize the results
result = task.result()
measurement = result.measurement_counts
print('measurement results:', measurement)

measurement results: Counter({'11': 401, '00': 394, '10': 103, '01': 102})


In [48]:
print(result)

GateModelQuantumTaskResult(task_metadata=TaskMetadata(braketSchemaHeader=BraketSchemaHeader(name='braket.task_result.task_metadata', version='1'), id='70daf071-a27e-486a-96a8-8adfc86c2d75', shots=1000, deviceId='braket_dm', deviceParameters=None, createdAt=None, endedAt=None, status=None, failureReason=None), additional_metadata=AdditionalMetadata(action=Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[2] b;\nqubit[2] q;\nh q[0];\n#pragma braket noise bit_flip(0.1) q[0]\ncnot q[0], q[1];\n#pragma braket noise bit_flip(0.1) q[0]\n#pragma braket noise bit_flip(0.1) q[1]\nb[0] = measure q[0];\nb[1] = measure q[1];', inputs={}), dwaveMetadata=None, ionqMetadata=None, rigettiMetadata=None, oqcMetadata=None, xanaduMetadata=None, queraMetadata=None, simulatorMetadata=None), result_types=[], values=[], measurements=array([[1, 1],
       [1, 1],
       [1, 1],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]]), meas

Questions:
1. what add measurement results mean?
2. 

In [38]:
def add_noise_suite(circ, noise_dict, measurements = {}):
    # # define a noise channel
    # noise = noises.BitFlip(probability=0.1)

    # # define a noise channel
    # noise = noises.Depolarizing(probability=0.1)
    
    for key in noise_dict.keys():
        dynamic_noise_func = getattr(noises, key)
        noise = dynamic_noise_func(noise_dict[key])

        # add noise to every gate in the circuit
        circ.apply_gate_noise(noise)
        measurement = noise_run(circ)
        measurements[key]=measurement

    return measurements


def noise_run(circ):
    # select the local noise simulator
    device = LocalSimulator('braket_dm')

    # run the circuit on the local simulator
    task = device.run(circ, shots = 1000)

    # visualize the results
    result = task.result()
    measurement = result.measurement_counts
    return measurement



In [44]:
# build a simple circuit
circ = Circuit().h(0).cnot(0,1)

noise_dict = {'BitFlip': 0.1, 'PhaseFlip': 0.1, 'Depolarizing': 0.1, 'AmplitudeDamping':0.1}
noise_dict = {'BitFlip': 0.1, 'PhaseFlip': 0.1, 'Depolarizing': 0.1, 'AmplitudeDamping':0.1, 'PhaseDamping':0.1}
#noise_dict = {'BitFlip': 0.1, 'PhaseFlip': 0.1, 'Depolarizing': 0.1, 'AmplitudeDamping':0.1, 'GeneralizedAmplitudeDamping': [0.1, 0.1], 'PhaseDamping':0.1, 'PauliChannel': [0.1, 0.1, 0.1]}

#using noise suite
measurements = add_noise_suite(circ, noise_dict)

for item in measurements.items():
    print(item)

('BitFlip', Counter({'11': 441, '00': 396, '01': 82, '10': 81}))
('PhaseFlip', Counter({'11': 425, '00': 399, '01': 88, '10': 88}))
('Depolarizing', Counter({'00': 369, '11': 364, '10': 136, '01': 131}))
('AmplitudeDamping', Counter({'00': 407, '11': 282, '01': 164, '10': 147}))
('PhaseDamping', Counter({'00': 393, '11': 293, '10': 178, '01': 136}))


In [43]:
dynamic_func = getattr(noises, 'Depolarizing')
print(dynamic_func(0.1))

Depolarizing(0.1)


In [32]:
noise = noises.Depolarizing(probability=0.1)
print(noise)

Depolarizing(0.1)
